In [1]:
import django_initializer

In [2]:
from ontrack.market.api.logic.endofdata import EndOfDayData
from ontrack.utils.datetime import DateTimeHelper as dt
from ontrack.utils.base.enum import ExchangeType, SettingKeyType
from ontrack.ta.utils._levels import get_eod_sr_levels, get_intraday_sr_levels
from ontrack.ta.core import sanitize
import ontrack.ta as ta
from operator import itemgetter
from ontrack.utils.numbers import NumberHelper as nh
from IPython.display import display, Markdown

import pandas as pd
import yfinance as yf
import matplotlib.dates as mpl_dates

In [3]:
import numpy as np
from pandas import Timestamp
import math
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.pyplot as plt

In [4]:
def get_stock_price(symbol, period="2y", interval="1d", start_date=None, end_date=None):
  df = yf.download(tickers=symbol, interval=interval, period=period, start=start_date, end=end_date)
  df['Date'] = pd.to_datetime(df.index)
  df['Date'] = df['Date'].apply(mpl_dates.date2num)
  df = df.loc[:,['Date', 'Open', 'High', 'Low', 'Close']]
  return df

def candle_stick(df):
    strategy = ta.Strategy(
        name="intraday",
        ta=[
            {
                "kind": "cdl_pattern",
                "name": "all",
                "consolidated": True,
                "append": True,
            },
            {"kind": "amat"},            
            {"kind": "cpr"},
            {"kind": "ema", "length": 5},
            {"kind": "ema", "length": 9},
            {"kind": "ema", "length": 13},
            {"kind": "bbands", "length": 20, "std": 1.5},
        ],
    )
    df.ta.cores = 0
    df.ta.sanitize()
    df.ta.strategy(strategy)

    row = df.ta.second_last_record

    cdl_rows = []
    cdl_string = row["CDL_CONSOLIDATED"].strip()
    value_ = ""
    if cdl_string:
        for cdl in cdl_string.split(";"):
            cdl_rs = cdl.split("|")
            name = cdl_rs[0]
            score = nh.str_to_float(cdl_rs[1])
            sentiment = "BULLISH" if score > 0 else "BEARISH"
            sentiment_html = f"<b style='color:#FF0000'>BEARISH</b> {name} {score}" if score < 0 else f"<b style='color:#008000'>BULLISH</b>  {name} {score}"

            cdl_row = {}
            cdl_row["rank"] = 0
            cdl_row["name"] = name
            cdl_row["sentiment"] = sentiment
            cdl_row["score"] = score
            cdl_rows.append(cdl_row)
            
            value_ = f"{value_}{sentiment_html}| "

    cdl_rows = sorted(cdl_rows, key=itemgetter("rank"), reverse=False)
    return cdl_string, cdl_rows, value_, row

def print_levels_data(record, price):
    print_value(f"#### {record['point'] - price}")
    print(f"Point: {record['point']}, Min: {record['min_point']}, Max: {record['max_point']}")
    print(f"Types: {set(record['types'])}")
    print("--------------------")

def print_result(levels, price):
    print("--------------------")
    price_printed = False
    
    for record in levels:
        if not record['is_below'] and not price_printed:
            display(Markdown(f"**Current Price: {price}**"))
            price_printed = True
            
        print_levels_data(record, price)
    
def print_value(message):
    display(Markdown(f"{message}"))
    
def print_cpr_location(price, prev_day):
    if price >= prev_day["CPR_R3"]:
        print_value("<b style='color:#FF0000'>ABOVE R3</b>")
    elif price >= prev_day["CPR_R2"]:
        print_value("<b style='color:#FF0000'>BETWEEN R2 AND R3</b>")
    elif price >= prev_day["CPR_R1"]:
        print_value("<span style='color:#ffa07a'>BETWEEN R2 AND R1</span>")
    elif price >= prev_day["CPR_TC"]:
        print_value("BETWEEN R1 AND TC")
    elif price >= prev_day["CPR_PIVOT"]:    
        print_value("BETWEEN PIVOT AND TC")
    elif prev_day["CPR_BC"] < price:
        print_value("BETWEEN PIVOT AND BC")
    elif prev_day["CPR_S1"] < price:
        print_value("<span style='color:#b2ec5d'>BETWEEN BC AND S1</span>")
    elif prev_day["CPR_S2"] < price:
        print_value("<b style='color:#b2ec5d'>BETWEEN S2 AND S1</b>")
    elif prev_day["CPR_S3"] < price:
        print_value("<b style='color:#008000'>BETWEEN S3 AND S2</b>")
    elif prev_day["CPR_S3"] > price:
        print_value("<b style='color:#008000'>BELOW S3")
        
def print_ema_location(price, high, low, ema, ema_name):
    if high <= ema:
        print_value(f"<span style='color:#b2ec5d'>Candle below {ema_name}</span>")
    elif low >= ema:
       print_value(f"<span style='color:#ffa07a'>Candle above {ema_name}<span>")
       
def print_bbband_location(price, upper, lower):
    if price <= lower:
        print_value(f"<span style='color:#b2ec5d'>price below BBAND</span>")
    elif price >= upper:
       print_value(f"<span style='color:#ffa07a'>price above BBAND</span>")
       
def print_analysis(df, name):
    r = candle_stick(df)
    print_value(f'### {name}:')
    print_value(f'{r[2]}')
    row = r[3]
    price = row["close"]
    high = row["high"]
    low = row["low"]
    
    print_value(f"AMAT Long Run: {row['AMATe_LR_8_21_2']}")
    print_value(f"AMAT Short Run: {row['AMATe_SR_8_21_2']}")
    
    ema = row["EMA_5"]
    print_ema_location(price, high, low, ema, "5 EMA")
    
    ema = row["EMA_9"]
    print_ema_location(price, high, low, ema, "9 EMA")
    
    ema = row["EMA_13"]
    print_ema_location(price, high, low, ema, "13 EMA")   
     
    upper = row["BBU_20_1.5"]
    lower = row["BBL_20_1.5"]
    print_bbband_location(price, upper, lower)
        


In [5]:
#symbol = 'TCS.NS'
#symbol = 'IDFCFIRSTB.NS'
#symbol = '^NSEBANK'
symbol = "^NSEI"
#symbol = "MSFT"
df_y = get_stock_price(symbol, "2y", "1d")
df_m = get_stock_price(symbol, "2y", "1mo")
df_w = get_stock_price(symbol, "2y", "1wk")
df_d = get_stock_price(symbol, "1y", "1d")
df_h = get_stock_price(symbol, "6mo", "1h")

df_y = sanitize(df_y)
df_m = sanitize(df_m)
df_w = sanitize(df_w)
df_d = sanitize(df_d)
df_h = sanitize(df_h)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
obj = EndOfDayData(ExchangeType.NSE)
result= obj.populate_indicators(df_y)
df = result[0]                
last_record = df.ta.second_last_record
eod_levels = get_eod_sr_levels(df, [df_m, df_w, df_h])

DEBUG 2022-11-19 18:03:59,267 logger 8596 9148 [c:\Sachin\OST-API\ontrack\utils\config.py.__get_config(16)]: Reading configuration [C:\Sachin\OST-API\config\app/urlconfig.json].


In [8]:
df_d = get_stock_price(symbol, "25d", "1d")
df_h = get_stock_price(symbol, "5d", "1h")
df_15m = get_stock_price(symbol, "3d", "15m")
df_5m = get_stock_price(symbol, "2d", "5m")
df_1m = get_stock_price(symbol, "1d", "1m")

df_d = sanitize(df_d)
df_h = sanitize(df_h)
df_15m = sanitize(df_15m)
df_5m = sanitize(df_5m)
df_1m = sanitize(df_1m)

open_ = df_d.iloc[-1]["open"]
price = df_15m.iloc[-1]["close"]

if (last_record["CPR"] * 1.5) < last_record["CPR_SMA_20"]:
    print_value("## NARROW CPR")

if price > open_:
    print_value("<b style='color:#b2ec5d'>Only Bullish in morning</b>")
else:
    print_value("<b style='color:#ffa07a'>Only Bearish in morning</b>")

print_value(f"## {symbol}")
print_analysis(df_d, "1 day")
print_cpr_location(price, last_record)
print_analysis(df_h, "1 hour")
print_analysis(df_15m, "15 minutes")
print_analysis(df_5m, "5 minutes")
print_analysis(df_1m, "1 minute")


levels = get_intraday_sr_levels(eod_levels, df_d, [df_5m, df_15m], price, item_count=10)
print_result(levels, price)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<b style='color:#ffa07a'>Only Bearish in morning</b>

## ^NSEI

### 1 day:

<b style='color:#FF0000'>BEARISH</b> CDL_HIGHWAVE -100.0| <b style='color:#FF0000'>BEARISH</b> CDL_SPINNINGTOP -100.0| 

...

Point: 18423.5, Min: 18423.5, Max: 18423.5
Types: {'SR_FCP'}
--------------------


#### 129.94960937499854

Point: 18437.6, Min: 18437.6, Max: 18442.15
Types: {'SR_WSM', '52W_H', 'C_W_H', 'C_M_H', 'SR_FCP'}
--------------------


#### 169.39960937499927

Point: 18477.05, Min: 18477.05, Max: 18488.82
Types: {'CPR_R2', 'ATC'}
--------------------
